In [225]:
#Import libraries
import pandas as pd
import re
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"

In [226]:
#Import the dataset
path='/Users/Dan/QUANTIUM/DATASETS'
df_transaction= pd.read_excel(path+"/QVI_transaction_data.xlsx")
print(df_transaction.head())

    DATE  STORE_NBR  LYLTY_CARD_NBR  TXN_ID  PROD_NBR  \
0  43390          1            1000       1         5   
1  43599          1            1307     348        66   
2  43605          1            1343     383        61   
3  43329          2            2373     974        69   
4  43330          2            2426    1038       108   

                                  PROD_NAME  PROD_QTY  TOT_SALES  
0    Natural Chip        Compny SeaSalt175g         2        6.0  
1                  CCs Nacho Cheese    175g         3        6.3  
2    Smiths Crinkle Cut  Chips Chicken 170g         2        2.9  
3    Smiths Chip Thinly  S/Cream&Onion 175g         5       15.0  
4  Kettle Tortilla ChpsHny&Jlpno Chili 150g         3       13.8  


In [227]:
# Sample half of the dataset
sampled_df = df_transaction.sample(frac=0.2, random_state=42)

# Print the first few rows of the sampled DataFrame
print(sampled_df.head())

         DATE  STORE_NBR  LYLTY_CARD_NBR  TXN_ID  PROD_NBR  \
147374  43365        219          219056  218335        13   
18693   43302         56           56203   51083        68   
98522   43533        262          262120  261994       112   
65042   43358        179          179013  179287        38   
176861  43391         95           95255   95244       113   

                                      PROD_NAME  PROD_QTY  TOT_SALES  
147374     Red Rock Deli Thai  Chilli&Lime 150g         2        5.4  
18693       Pringles Chicken    Salt Crips 134g         2        7.4  
98522    Tyrrells Crisps     Ched & Chives 165g         2        8.4  
65042   Infuzions Mango     Chutny Papadums 70g         2        4.8  
176861                     Twisties Chicken270g         2        9.2  


In [228]:
#Let's see what the data looks like
sampled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52967 entries, 147374 to 98614
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DATE            52967 non-null  int64  
 1   STORE_NBR       52967 non-null  int64  
 2   LYLTY_CARD_NBR  52967 non-null  int64  
 3   TXN_ID          52967 non-null  int64  
 4   PROD_NBR        52967 non-null  int64  
 5   PROD_NAME       52967 non-null  object 
 6   PROD_QTY        52967 non-null  int64  
 7   TOT_SALES       52967 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 3.6+ MB


In [229]:
#Let's see the NaN or null values in the dataframe and evaluate if they affect the  Analysis
sampled_df.isna().sum()

DATE              0
STORE_NBR         0
LYLTY_CARD_NBR    0
TXN_ID            0
PROD_NBR          0
PROD_NAME         0
PROD_QTY          0
TOT_SALES         0
dtype: int64

In [230]:
#Let's convert 'Date' to datetime
from datetime import datetime, timedelta
base_date = pd.to_datetime('1899-12-30')
sampled_df['DATE'] =base_date+pd.to_timedelta(sampled_df['DATE'], unit='D')
sampled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52967 entries, 147374 to 98614
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATE            52967 non-null  datetime64[ns]
 1   STORE_NBR       52967 non-null  int64         
 2   LYLTY_CARD_NBR  52967 non-null  int64         
 3   TXN_ID          52967 non-null  int64         
 4   PROD_NBR        52967 non-null  int64         
 5   PROD_NAME       52967 non-null  object        
 6   PROD_QTY        52967 non-null  int64         
 7   TOT_SALES       52967 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(5), object(1)
memory usage: 3.6+ MB


In [231]:
#Let's check the unique values of column PROD_NAME 
unique_values = sampled_df['PROD_NAME'].unique()
print(unique_values)

['Red Rock Deli Thai  Chilli&Lime 150g'
 'Pringles Chicken    Salt Crips 134g'
 'Tyrrells Crisps     Ched & Chives 165g'
 'Infuzions Mango     Chutny Papadums 70g' 'Twisties Chicken270g'
 'Cheezels Cheese Box 125g' 'Doritos Salsa Mild  300g'
 'Grain Waves Sour    Cream&Chives 210G' 'Twisties Cheese     Burger 250g'
 'Smiths Crinkle Cut  Chips Chs&Onion170g'
 'Thins Chips Seasonedchicken 175g'
 'Old El Paso Salsa   Dip Tomato Med 300g'
 'Smiths Crinkle Cut  Chips Original 170g'
 'Dorito Corn Chp     Supreme 380g' 'Doritos Cheese      Supreme 330g'
 'Old El Paso Salsa   Dip Tomato Mild 300g'
 'Doritos Corn Chips  Cheese Supreme 170g'
 'Kettle Tortilla ChpsHny&Jlpno Chili 150g'
 'NCC Sour Cream &    Garden Chives 175g'
 'Kettle Sweet Chilli And Sour Cream 175g'
 'Smiths Crinkle Cut  Tomato Salsa 150g'
 'Smiths Chip Thinly  Cut Original 175g'
 'Cobs Popd Swt/Chlli &Sr/Cream Chips 110g'
 'WW Sour Cream &OnionStacked Chips 160g' 'Pringles SourCream  Onion 134g'
 'WW D/Style Chip     Sea Salt

In [232]:
# Function to remove digits and special characters
#def remove_digits_special_chars(text):
    # Using regular expression to remove digits and special characters
    #return re.sub(r'[^a-zA-Z\s]', '', text)

# Apply the function to the column
#sampled_df['PROD_NAME'] = sampled_df['PROD_NAME'].apply(remove_digits_special_chars)

#print(sampled_df['PROD_NAME'].head(10))

In [233]:
# Split text into words and count occurrences
word_counts = sampled_df['PROD_NAME'].str.split().explode().value_counts()

# Display the top 25 repeating words
print(word_counts.head(25))

175g        12049
Chips       10056
150g         8244
Kettle       8205
&            7085
Smiths       5912
Salt         5642
Cheese       5609
Pringles     5055
134g         5055
Doritos      4969
Crinkle      4936
110g         4517
Corn         4415
Original     4353
Cut          4221
170g         3744
Chip         3680
Salsa        3621
Chicken      3067
165g         3044
Sea          2816
Sour         2811
Thins        2745
Chilli       2572
Name: PROD_NAME, dtype: int64


In [234]:
# Word to remove rows containing
word_to_remove = 'Salsa'

# Filter rows that do not contain the word
filtered_df = sampled_df[~sampled_df['PROD_NAME'].str.contains(word_to_remove)]
print(filtered_df.head())

             DATE  STORE_NBR  LYLTY_CARD_NBR  TXN_ID  PROD_NBR  \
147374 2018-09-22        219          219056  218335        13   
18693  2018-07-21         56           56203   51083        68   
98522  2019-03-09        262          262120  261994       112   
65042  2018-09-15        179          179013  179287        38   
176861 2018-10-18         95           95255   95244       113   

                                      PROD_NAME  PROD_QTY  TOT_SALES  
147374     Red Rock Deli Thai  Chilli&Lime 150g         2        5.4  
18693       Pringles Chicken    Salt Crips 134g         2        7.4  
98522    Tyrrells Crisps     Ched & Chives 165g         2        8.4  
65042   Infuzions Mango     Chutny Papadums 70g         2        4.8  
176861                     Twisties Chicken270g         2        9.2  


In [235]:
# Search for word Salsa to confirm it was removed in the 'PROD_NAME' column
search_word = 'Salsa'
result = filtered_df[filtered_df['PROD_NAME'] == search_word]
print(result)

Empty DataFrame
Columns: [DATE, STORE_NBR, LYLTY_CARD_NBR, TXN_ID, PROD_NBR, PROD_NAME, PROD_QTY, TOT_SALES]
Index: []


In [236]:
filtered_df.isna().sum()

DATE              0
STORE_NBR         0
LYLTY_CARD_NBR    0
TXN_ID            0
PROD_NBR          0
PROD_NAME         0
PROD_QTY          0
TOT_SALES         0
dtype: int64

In [237]:
#Get statistical summary of the data
filtered_df.describe()

,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_QTY,TOT_SALES
count,49346.000000,4.934600e+04,49346.000000,49346.000000,49346.000000,49346.000000
mean,135.164694,1.355692e+05,135222.314048,56.407267,1.912495,7.335796
std,76.842695,7.965836e+04,78074.144427,33.690797,1.305919,4.782855
min,1.000000,1.011000e+03,14.000000,1.000000,1.000000,1.700000
25%,70.000000,7.003400e+04,67686.500000,26.000000,2.000000,5.800000
50%,130.000000,1.303740e+05,135197.500000,53.000000,2.000000,7.400000
75%,203.000000,2.031182e+05,202769.250000,87.000000,2.000000,8.800000
max,272.000000,2.370581e+06,270208.000000,114.000000,200.000000,650.000000


In [238]:
#Outlier detection
upper_outlier=200
rows_w_outlier=filtered_df[filtered_df['PROD_QTY'] == upper_outlier]
mask = filtered_df['PROD_QTY'] != upper_outlier
new_df=filtered_df[mask]
new_df.describe()
print(new_df.head(5))

             DATE  STORE_NBR  LYLTY_CARD_NBR  TXN_ID  PROD_NBR  \
147374 2018-09-22        219          219056  218335        13   
18693  2018-07-21         56           56203   51083        68   
98522  2019-03-09        262          262120  261994       112   
65042  2018-09-15        179          179013  179287        38   
176861 2018-10-18         95           95255   95244       113   

                                      PROD_NAME  PROD_QTY  TOT_SALES  
147374     Red Rock Deli Thai  Chilli&Lime 150g         2        5.4  
18693       Pringles Chicken    Salt Crips 134g         2        7.4  
98522    Tyrrells Crisps     Ched & Chives 165g         2        8.4  
65042   Infuzions Mango     Chutny Papadums 70g         2        4.8  
176861                     Twisties Chicken270g         2        9.2  


In [239]:
#get number of transactions by date
transaction_counts = new_df.groupby('DATE').size().reset_index(name='transaction_count')
print(transaction_counts)

          DATE  transaction_count
0   2018-07-01                139
1   2018-07-02                115
2   2018-07-03                129
3   2018-07-04                117
4   2018-07-05                150
..         ...                ...
359 2019-06-26                141
360 2019-06-27                137
361 2019-06-28                130
362 2019-06-29                138
363 2019-06-30                126

[364 rows x 2 columns]


In [240]:
# Create a sequence of dates
dates = pd.date_range(start='2018-07-01', end='2019-06-30', freq='D')
# Create a DataFrame with these dates
date_df = pd.DataFrame({'DATE': dates})
print(date_df)
# Merge date_df with your original DataFrame on the 'date' column
merged_df = pd.merge(date_df,new_df, on='DATE', how='left')
# Count the number of transactions for each date
transaction_counts = merged_df.groupby('DATE').size()
# Create a Plotly trace
trace = go.Scatter(x=transaction_counts.index, y=transaction_counts.values, mode='lines', name='Number of Transactions')

# Create layout
layout = go.Layout(title='Number of Transactions Over Time', xaxis=dict(title='Date'), yaxis=dict(title='Number of Transactions'))

# Create Figure object
fig = go.Figure(data=[trace], layout=layout)

# Show the plot
fig.show()



          DATE
0   2018-07-01
1   2018-07-02
2   2018-07-03
3   2018-07-04
4   2018-07-05
..         ...
360 2019-06-26
361 2019-06-27
362 2019-06-28
363 2019-06-29
364 2019-06-30

[365 rows x 1 columns]


In [241]:
transaction_counts = merged_df.groupby('DATE').size().reset_index(name='transaction_count')

december_data = transaction_counts[(transaction_counts['DATE'] >= '2018-12-01') & (transaction_counts['DATE'] <= '2018-12-31')]

scatter_plot = go.Scatter(x=december_data['DATE'], y=december_data['transaction_count'], mode='lines', name='Number of Transactions')

# Create layout with x-axis range limited to December
layout = go.Layout(title='Number of Transactions in December 2018', xaxis=dict(title='Date', range=['2018-12-01', '2018-12-31']), yaxis=dict(title='Number of Transactions'))

# Create Figure object
fig = go.Figure(data=[scatter_plot], layout=layout)

# Show the plot
fig.show()


In [242]:
#get the pack size
pack_size= merged_df['PROD_NAME'].str.extract(r'(\d+)').astype(float)
merged_df['PACK_SIZE']=pack_size
print(merged_df.head())


        DATE  STORE_NBR  LYLTY_CARD_NBR    TXN_ID  PROD_NBR  \
0 2018-07-01       35.0         35052.0   31630.0      11.0   
1 2018-07-01      116.0        116179.0  120232.0      81.0   
2 2018-07-01       38.0         38324.0   34376.0      79.0   
3 2018-07-01      106.0        106135.0  107555.0      81.0   
4 2018-07-01      210.0        210159.0  209607.0      63.0   

                                PROD_NAME  PROD_QTY  TOT_SALES  PACK_SIZE  
0                RRD Pc Sea Salt     165g       1.0        3.0      165.0  
1         Pringles Original   Crisps 134g       2.0        7.4      134.0  
2  Smiths Chip Thinly  CutSalt/Vinegr175g       2.0        6.0      175.0  
3         Pringles Original   Crisps 134g       2.0        7.4      134.0  
4            Kettle 135g Swt Pot Sea Salt       2.0        8.4      135.0  


In [243]:
# Get the highest value of pack size
highest_value = merged_df['PACK_SIZE'].max()

# Get the lowest value of pack size
lowest_value = merged_df['PACK_SIZE'].min()

print("Highest value:", highest_value)
print("Lowest value:", lowest_value)

Highest value: 380.0
Lowest value: 70.0


In [248]:
# Count the number of transactions for each pack size
pack_transactions = merged_df.groupby('PACK_SIZE').size().reset_index(name='TRANSACTIONS')

# Create bar chart
fig = go.Figure(data=[go.Bar(x=pack_transactions['PACK_SIZE'], y=pack_transactions['TRANSACTIONS'])])

# Update layout
fig.update_layout(
    title="Number of Transactions by Pack Size",
    xaxis_title="Pack Size",
    yaxis_title="Transactions"
)

# Show the plot
fig.show()